In [1]:
import TABprocessor
from TABprocessor import *
import os
path = os.getcwd()

In [3]:
#chord_recognize accurate test

songs = readTAB(path + '\\test\\chord_recognize test\\')
cp, cf, ct = [song.pitch for song in songs], [song.finger for song in songs], [song.time for song in songs]
recognize_result = []
_ = []
for chord in split(cp[0]):
    _.append(chord_recognize(chord))
    
    if len(_) == 8:
        recognize_result.append(_)
        _ = []

        

#####0. chord_recognize test.xml
    ! In measure 16, sum of timesig should be 4.0, but0


In [4]:
import pandas as pd
yes, no, sus_ignore_no = 0, 0, 0
truth = pd.read_csv('chord_recognize test.csv')
for num1, measure in enumerate(recognize_result):
    print('measure {}:...'.format(num1))
    for num2, chord in enumerate(measure):
        
        real = truth.iloc[num1][num2]
        root, _type = 0, 0
        if '#' in real:
            root = real[:2]
            _type = real[2:]
        else:
            root = real[:1]
            _type = real[1:]
        if _type == '':
            _type = 'maj'
        #print("/{}/, {}".format(root, _type))
        if real != 'N' and (chord[0] + chord[1] != root+_type):
            no += 1
            sus_ignore_no += 1
            print('measure{}, note{} wrong: should `` {} ``, but`` {} ``'.format(num1, num2, root+_type, chord[0] + chord[1]))
            if 'sus' in _type and 'sus' in chord[1]:
                sus_ignore_no -= 1
        elif chord[0] == root and chord[1] == _type and real != 'N':
            yes += 1
    print('=================================')
    print()
            
 

measure 0:...

measure 1:...

measure 2:...
measure2, note4 wrong: should `` Gsus ``, but`` Csus ``

measure 3:...

measure 4:...

measure 5:...
measure5, note7 wrong: should `` Gsus ``, but`` NotChordNotChord ``

measure 6:...

measure 7:...
measure7, note2 wrong: should `` Em7 ``, but`` NotChordNotChord ``

measure 8:...
measure8, note0 wrong: should `` Asus ``, but`` NotChordNotChord ``
measure8, note3 wrong: should `` Dsus ``, but`` Gsus ``

measure 9:...

measure 10:...
measure10, note3 wrong: should `` Dsus ``, but`` Gsus ``

measure 11:...

measure 12:...
measure12, note6 wrong: should `` Bsus4 ``, but`` NotChordNotChord ``

measure 13:...

measure 14:...
measure14, note4 wrong: should `` Asus ``, but`` NotChordNotChord ``

measure 15:...
measure15, note5 wrong: should `` Asus ``, but`` NotChordNotChord ``



In [5]:
print('acc: {}%'.format(100*yes/(yes+no)))
print('sus-ignored acc: {:.2f}%'.format(100*(yes+no-sus_ignore_no)/(yes+no)))

acc: 92.5%
sus-ignored acc: 95.00%


In [6]:
def chord_recognize(pitch_cu, return_overtone=False):
    '''
    大三和弦：有且仅有大三度和五度， 435 435 435 或 75 或 48 39
    小三和弦：有且仅有小三度和五度， 345 345 345
    大七和弦：存在大七度和大三度， 4341 4341 4341或471
    小七和弦：存在小七度和小三度， 3432 3432 3432或732
    属七和弦：存在小七度和大三度， 4332 4332 4332或732
    增三和弦：存在增五度（小六度）， 444 444 444
    减三和弦：存在增四度， 326 326 326或
    挂留2和弦：存在二度但不存在三度， 255 255 255
    挂留4和弦：存在四度但不存在三度， 525 525 525
    '''
    in_chord_pitchs = [i[1:] for i in pitch_cu.split()][-4:]
    pitchs = list(set(in_chord_pitchs))
    singles = list(np.sort([str2midi(i) for i in pitchs]))
    print(singles)
    _ = singles
    singles.extend([i + 12 for i in _])
    singles.extend([i + 24 for i in _])
    
    delta_singles = []
    for i in range(len(singles)):
        if i > 0:
            delta_singles.append(singles[i] - singles[i-1])
    inverts = delta_singles
    feasure = ''
    for num in delta_singles:
        feasure += str(num)
    def ifin(str1):
        strlen = len(str1)
        newstrs = []
        for i in range(len(str1)):
            str1 = str1[1:] + str1[0]
            newstrs.append(str1)
        for newstr in newstrs:
            if newstr in feasure:
                return True
        return False
    print(feasure)
    chord_type = 'NotChord'
    chordchar = 'NotChord'
    if ifin('255'): #当一阶差分只有5半音和2半音时 说明是挂2
        chord_type = 'sus2'
        
        chordchar = midi2str(singles[feasure.index('2')])[1:]#2 前面的音符是挂2的根音

    elif ifin('336'):
        chord_type =  'dim'
        chordchar = midi2str(singles[feasure.index('6')+1])[1:]#6后面的是根音
    elif ifin('444'): #aug时返回最低的音
        chord_type = 'aug'
        chordchar = in_chord_pitchs[0]
    #七和弦组：
    
    if ifin('2') or ifin('1'): 
        #属七：
        if ifin('4332') or ifin('246'):
            chord_type = '7'
            chordchar = midi2str(singles[feasure.index('4')])[1:] #4前面
        #小七：              
        elif ifin('3432') or ifin('723') or ifin('732'):
            chord_type = 'm7'
            chordchar = midi2str(singles[feasure.index('2')+1])[1:] #2后面
        elif ifin('4341') or ifin('471') or ifin('741'):
            chord_type = 'maj7'
            chordchar = midi2str(singles[feasure.index('1')+1])[1:] #1后面
    #三和弦组：
    else:
        if ifin('345') or ifin('39'):
            chord_type = 'm'
            chordchar = midi2str(singles[feasure.index('3')])[1:] #3前面
        elif ifin('435') or ifin('75'):
            chord_type = 'maj'
            chordchar = midi2str(singles[feasure.index('5')+1])[1:] #5后面
        elif ifin('48') :
            chord_type = 'maj'
            chordchar = midi2str(singles[feasure.index('4')])[1:] #4前面
    if return_overtone == True:
        return [chord_type, singles]
    else:
        return chordchar, chord_type
    
  

In [7]:
def root_detect(measure, tune=[0, 0, 0, 0, 0, 0], capo=0, clear_zero=True):
    '''
    根音和旋律音检测
    measure:一个三个元素的list，分别是小节内的所有音、指法、时间
    '''
    pitchs, fingers, times = measure[0].copy(), measure[1].copy(), measure[2].copy() #分别代表音高、指法、时间

    pitchnum = len(pitchs)
    indexlist = list(np.zeros(pitchnum))
    #print('待检测的音为{}'.format(pitchs))
    roots = [] 
    melodys = []
    root_thr = 5
    
    #设置根音门槛
    allfingers = []
    for finger in fingers:
        allfingers.extend(finger.split())
    rootcount = 0
    for finger in allfingers:
        if ('R' not in finger) and int(finger[1]) >= 5:
            rootcount += 1
    if rootcount <= 1:
        root_thr = 4


    for i in range(pitchnum):  #该小节内的第i个音
        
        multi = 0
        pitch_cu = pitchs[i].split()
        finger_cu = fingers[i].split()
        if len(pitch_cu) >= 2:
            multi = 1
        melody, root = 0, 0
        if 'R' not in finger_cu[0]:
            for k in range(len(finger_cu)):
                finger_cu[k] = midi2finger(finger2midi(finger_cu[k],tune=tune,capo=capo), tune=tune, capo=capo)[-1]
            #print('note {}: {}, thr={}'.format(i, finger_cu, root_thr))
            if (int(finger_cu[0][1]) <= 3) or (multi == 1) :  #当音簇最高音在1或2弦，或者音簇为多音时
                #melody = finger2midi(finger_cu[0])  
                indexlist[i] += 10
                melody = str2midi(pitch_cu[0])
                #print('    旋律音:{}, 其midi值为{}, 在{}'.format(midi2str(int(melody)), melody, finger_cu[0]))
            if (int(finger_cu[-1][1]) >= root_thr):  #当音簇最低音在5或6弦时
                root = str2midi(pitch_cu[-1]) 
                indexlist[i] += 5
                #print('    根音:{}, 其midi值{}, 在{}'.format(midi2str(int(root)), root, finger_cu[-1]))
            #print('---------------')
        # roots.append(midi2str(int(root)))      
        # melodys.append(midi2str(int(melody)))
    #return [melodys, roots]  #返回该小节内所有满足条件的根音和旋律音
    
    for i in range(pitchnum):
        if indexlist[i] == 15:#当根音旋律音都有
            pitchs[i] = pitchs[i].split()[0] + ' '+pitchs[i].split()[-1]
            fingers[i] = fingers[i].split()[0] + ' '+fingers[i].split()[-1]
        elif indexlist[i] == 10:#只有旋律音
            pitchs[i] = pitchs[i].split()[0]
        elif indexlist[i] == 5:#只有根音
            fingers[i] = fingers[i].split()[-1]
    if indexlist[0] == 0:
        pitchs[0], fingers[0] = 'R', '(R,R)'
    for i in range(len(times)-1, -1, -1):
        if indexlist[i] == 0 and i >0:#当二者都没有时：
            times[i-1] += times[i]
            times[i] = 0 #将该时值交给前面
            pitchs[i], fingers[i] = '', ''
    #最后清除空元素
    if clear_zero:
        for i in range(len(times)-1, -1, -1):
            if times[i] == 0:
                pitchs.pop(i)
                fingers.pop(i)
                times.pop(i)
    return [pitchs, fingers, times]

In [9]:
#bassline detect test
path2 = path + '\\test\\melody extraction test\\'

songs = readTAB(path2)
testp, testf, testt = [song.pitch for song in songs], [song.finger for song in songs], [song.time for song in songs]

#####0. [EVA]残酷的天使纲领.xml
#####1. [初音未来]Letter Song(致十年后的我).xml
capo 3
#####2. [千与千寻]与你同在.xml
    ! In measure 0, sum of timesig should be 1.5, but3.0
    ! In measure 1, sum of timesig should be 1.5, but3.0
    ! In measure 2, sum of timesig should be 1.5, but3.0
    ! In measure 3, sum of timesig should be 1.5, but3.0
    ! In measure 4, sum of timesig should be 1.5, but3.0
    ! In measure 5, sum of timesig should be 1.5, but3.0
    ! In measure 6, sum of timesig should be 1.5, but3.0
    ! In measure 7, sum of timesig should be 1.5, but3.0
    ! In measure 8, sum of timesig should be 1.5, but3.0
    ! In measure 9, sum of timesig should be 1.5, but3.0
    ! In measure 10, sum of timesig should be 1.5, but3.0
    ! In measure 11, sum of timesig should be 1.5, but3.0
    ! In measure 12, sum of timesig should be 1.5, but3.0
    ! In measure 13, sum of timesig should be 1.5, but3.0
    ! In measure 14, sum of timesig should be 1.5, but3.0
    ! In measure 15, sum of timesig should be 1

In [10]:
testing = []
for i in range(len(testp[1])):
    testing.append([testp[1][i], testf[1][i], testt[1][i]])
    
result = []
for measure in testing:
    result.append(root_detect(measure, capo=3))
truth = []
for q in zip(testp[5], testf[5], testt[5]):
    truth.append(list(q))
print(result[5])
print(truth[5])

[['4F 3D', '3F', '4F', '4E', '3F', '4F', '4F'], ['(1,0) (5,2)', '(4,0)', '(1,0)', '(2,4)', '(4,0)', '(1,0)', '(1,0)'], [0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5]]
[['4F 3D', '4F', '4E', '3F', '4F', '4F'], ['(1,0) (5,2)', '(1,0)', '(2,4)', '(4,0)', '(1,0)', '(1,0)'], [1.0, 0.5, 0.5, 0.5, 1.0, 0.5]]


In [11]:
answerdict = {
        'right':0, 
        'extra_root':0, 
        'extra_melody':0, 
        'miss_root':0, 
        'miss_melody':0, 
        'all_miss':0, 
        'all_extra':0, 
        'reverse':0
    }
rootdict = {
    'TP':0, #真值有root，识别出root
    'FP':0, #真值无，识别出root
    'TN':0, #真值无，未识别
    'FN':0 #真值有root，未识别
    }
rootname = ['TN', 'FN', 'FP', 'FN', 'TN', 'TN', 'TP', 'FP',
            'FN', 'TP', 'TP', 'FP', 'FN', 'TN', 'TP', 'FP']

melodydict = {
    'TP':0, #真值有MELODY，识别出MELODY
    'FP':0, #真值无，识别出MELODY
    'TN':0, #真值无，未识别
    'FN':0 #真值有MELODY，未识别
    }
melodyname = ['TN', 'TN', 'TN', 'FN', 'FP', 'FN', 'TN', 'FP',
              'FP', 'FN', 'FP', 'FN', 'TP', 'TP', 'TP', 'TP']
answer = [2,4,6,8,10,11,12,14,20,24,28,33,40,55,56,57]
answername = ['right', 'miss_root', 'extra_root', 'all_miss',
              'extra_melody', 'miss_melody', 'right', 'all_extra',
              'reverse', 'miss_melody', 'extra_melody', 'reverse',
              'miss_root', 'right', 'right', 'extra_root']
for measurenum, measure in enumerate(list(zip(result[:len(truth)], truth))):
    #print('measure {}...'.format(measurenum))
    test_rootnum = len([finger for finger in measure[0][1] if 'R' not in finger and int(finger.split()[-1][1]) >3])
    real_rootnum = len([finger for finger in measure[1][1] if 'R' not in finger and int(finger.split()[-1][1]) >3])
    right_root, miss_root, extra_root = 0, 0, 0
    testpitchnum, realpitchnum = len(measure[0]), len(measure[1])
    #该小节的第一部分是测试结果，第二部分是答案
    measure[0][2] = list(np.cumsum(measure[0][2]) - measure[0][2])
    measure[1][2] = list(np.cumsum(measure[1][2]) - measure[1][2])#由时值长度变为绝对时间坐标
    
    totaltime = measure[0][2]
    totaltime.extend(measure[1][2])
    totaltime = sorted(list(set(totaltime)))
    testlist = np.ones(len(totaltime))
    truthlist = 2 * testlist.copy()
    for i, finger_cu in enumerate(measure[0][1]):
        _ = totaltime.index(measure[0][2][i])
        #用4代表根音， 5代表旋律音
        if 'R' not in finger_cu:
            if int(finger_cu.split()[-1][1]) >3:#如果是根音的话
                testlist[_] = 3
            if int(finger_cu.split()[0][1]) <3:#如果是旋律音的话
                testlist[_] = 5
            if len(finger_cu.split()) >1:#如果都有
                testlist[_] = 7
    for i, finger_cu in enumerate(measure[1][1]):
        _ = totaltime.index(measure[1][2][i])
        if 'R' not in finger_cu:
            if int(finger_cu.split()[-1][1]) >3:#如果是根音的话
                truthlist[_] = 4
            if int(finger_cu.split()[0][1]) <3:#如果是旋律音的话
                truthlist[_] = 11
            if len(finger_cu.split()) >1:#如果都有
                truthlist[_] = 8

    multilist = np.array(testlist) * np.array(truthlist)
    #print(multilist)
    for number in multilist:
        key = answername[answer.index(number)]
        answerdict[key] += 1
    for number in multilist:
        key = rootname[answer.index(number)]
        rootdict[key] += 1
    for number in multilist:
        key = melodyname[answer.index(number)]
        melodydict[key] += 1





print(answerdict)
print(rootdict)
print(melodydict)

{'right': 254, 'extra_root': 39, 'extra_melody': 14, 'miss_root': 8, 'miss_melody': 0, 'all_miss': 0, 'all_extra': 1, 'reverse': 0}
{'TP': 76, 'FP': 40, 'TN': 192, 'FN': 8}
{'TP': 183, 'FP': 15, 'TN': 118, 'FN': 0}


In [12]:
root_precision = rootdict['TP'] / (rootdict['TP'] + rootdict['FP'])
root_recall = rootdict['TP'] / (rootdict['TP'] + rootdict['FN'])
melody_precision = melodydict['TP'] / (melodydict['TP'] + melodydict['FP'])
melody_recall = melodydict['TP'] / (melodydict['TP'] + melodydict['FN'])
print('root_precison: {:.2f}%'.format(100 * root_precision))
print('root_recall: {:.2f}%'.format(100 * root_recall))
print('melody_precison: {:.2f}%'.format(100 * melody_precision))
print('melody_recall: {:.2f}%'.format(100 * melody_recall))

root_precison: 65.52%
root_recall: 90.48%
melody_precison: 92.42%
melody_recall: 100.00%
